<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/multi_phase_self_destruct_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
multi_phase_self_destruct_ai.py

Works in terminals *and* Jupyter/Colab by ignoring unknown CLI args.
"""

import torch
import torch.nn as nn
import random
import argparse
import pprint

class MultiPhaseSelfDestructAI(nn.Module):
    def __init__(
        self,
        init_state: float = 1e6,
        phases: int = 5,
        decay: str = "linear",
        random_death_prob: float = 0.0,
    ):
        super().__init__()
        self.register_buffer("null_state", torch.tensor(init_state, dtype=torch.float32))
        self.init_state = float(init_state)
        self.phases = phases
        self.decay = decay
        self.random_death_prob = random_death_prob
        self.current_phase = 0
        self.alive = True
        self._logs = []

    def forward(self, x: torch.Tensor):
        if not self.alive:
            return None

        # Random-triggered collapse
        if random.random() < self.random_death_prob:
            self._snapshot("random collapse triggered")
            self._self_destruct()
            return None

        out = x * self.null_state
        self._snapshot(f"before phase {self.current_phase + 1}")
        self.current_phase += 1

        with torch.no_grad():
            if self.decay == "linear":
                delta = self.init_state / self.phases
                new_val = float(self.null_state) - delta
                self.null_state.fill_(max(new_val, 0.0))
            else:
                self.null_state.mul_(0.5)

        if self.current_phase >= self.phases or float(self.null_state) == 0.0:
            self._snapshot("natural end of life")
            self._self_destruct()

        return out

    def _snapshot(self, note: str):
        self._logs.append({
            "phase": self.current_phase,
            "null_state": float(self.null_state),
            "note": note,
        })

    def _self_destruct(self):
        key = random.getrandbits(128)
        encrypted = repr(self._logs).encode("utf-8")[::-1]
        del key, encrypted, self._logs
        self.alive = False

    @property
    def logs(self):
        return self._logs if self.alive else []

def run_demo(init, phases, decay, rand):
    model = MultiPhaseSelfDestructAI(init, phases, decay, rand)
    sample = torch.tensor([init], dtype=torch.float32)

    print("=== Multi-Phase Self-Destruct AI Demo ===")
    print(f"Initial null_state = {model.null_state.item():.3e}")
    print(f"Decay strategy   = {decay}")
    print(f"Phases           = {phases}")
    print(f"Random death     = {rand:.2%}\n")

    step = 0
    while True:
        step += 1
        out = model(sample)
        if out is None:
            print(f"Step {step:3d}: model has self-destructed.")
            break
        print(
            f"Step {step:3d}: output = {out.item():.3e} | "
            f"null_state = {model.null_state.item():.3e}"
        )
        if step > phases + 5:
            print("Max steps reached, forcing exit.")
            break

    print("\nFinal logs (should be empty):")
    pprint.pprint(model.logs)
    del model
    print("\nAll traces removed. Goodbye.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Run a multi-phase self-destructing AI demo"
    )
    parser.add_argument("--init", type=float, default=1e6)
    parser.add_argument("--phases", type=int, default=5)
    parser.add_argument("--decay", choices=["linear","exponential"], default="linear")
    parser.add_argument("--rand", type=float, default=0.0)
    # Ignore any extra args (e.g. Jupyter’s -f flag)
    args, _ = parser.parse_known_args()
    run_demo(args.init, args.phases, args.decay, args.rand)